In [94]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn.metrics as metrics
import seaborn as sns
from sklearn.model_selection import train_test_split

In [134]:
def train_model_pipeline(dataset_location, lr=0.005, epochs=100):
    # create image dataset from directory
    def create_dataset(directory, subset):
        return tf.keras.preprocessing.image_dataset_from_directory(
            directory,
            seed=123,
            image_size=(48, 128),
            label_mode='binary',
            shuffle=True,
            validation_split=0.3,
            subset=subset
        )

    # create a model using simple CNN for binary classification
    def create_model(num_channels):
        model = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(48, 128, num_channels), name="Rescale_input"),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name="Conv1"),
            tf.keras.layers.MaxPooling2D((2, 2), name="MaxPool1"),
            tf.keras.layers.Conv2D(64, (3, 3), activation='relu', name="Conv2"),
            tf.keras.layers.MaxPooling2D((2, 2), name="MaxPool2"),
            tf.keras.layers.Conv2D(128, (3, 3), activation='relu', name="Conv3"),
            tf.keras.layers.MaxPooling2D((2, 2), name="MaxPool3"),
            tf.keras.layers.Flatten(name="Flatten_features"),
            tf.keras.layers.Dense(128, activation='relu', name="FC1"),
            tf.keras.layers.Dense(1, activation='sigmoid', name="Output")
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy', 'AUC']
        )
        return model

    # train model
    def train_model(model, train, validation):
        history = model.fit(
            train,
            validation_data=validation,
            epochs=epochs,
            callbacks=[tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', min_delta=0.001, patience=10, verbose=1, restore_best_weights=True)]
        )
        return history

    # create image datasets from directory
    dataset_train = create_dataset(dataset_location, subset="training")
    dataset_val = create_dataset(dataset_location, subset="validation").shuffle(1185)

    # # Split val into val and test
    # test_size = 0.5
    # val_batches = dataset_val.cardinality().numpy()
    # test_batches = int(test_size * val_batches)
    # dataset_test = dataset_val.take(test_batches)
    # dataset_val = dataset_val.skip(test_batches)

    # create model
    model = create_model(3)

    # train model
    history = train_model(model, dataset_train, dataset_val)

    # Extract true labels from the test set
    true_labels = []
    for _, labels in dataset_val:
        true_labels.append(labels.numpy()[0])

    # Predict on the test set
    test_pred = model.predict(dataset_val)
    test_pred_class = (test_pred > 0.5).astype('int32')

    eval_dict = {
        "test_proba": test_pred,
        "test_pred_class": test_pred_class,
        "true_labels": true_labels
    }

    return model, history, eval_dict


In [135]:
# train using mel
stft_model, stft_history, stft_eval_dict = train_model_pipeline('data/organized_spectrograms/STFT', lr=0.005, epochs=20)

Found 3951 files belonging to 2 classes.
Using 2766 files for training.
Found 3951 files belonging to 2 classes.
Using 1185 files for validation.
Epoch 1/20
87/87 [==============================] - 3s 24ms/step - loss: 0.4662 - accuracy: 0.7683 - auc: 0.8232 - val_loss: 0.3206 - val_accuracy: 0.8903 - val_auc: 0.9305
Epoch 2/20
87/87 [==============================] - 2s 23ms/step - loss: 0.3641 - accuracy: 0.8369 - auc: 0.9032 - val_loss: 0.3639 - val_accuracy: 0.8920 - val_auc: 0.9321
Epoch 3/20
87/87 [==============================] - 2s 20ms/step - loss: 0.3290 - accuracy: 0.8706 - auc: 0.9255 - val_loss: 0.3861 - val_accuracy: 0.7806 - val_auc: 0.9386
Epoch 4/20
87/87 [==============================] - 2s 21ms/step - loss: 0.2778 - accuracy: 0.8897 - auc: 0.9366 - val_loss: 0.3157 - val_accuracy: 0.8489 - val_auc: 0.9459
Epoch 5/20
87/87 [==============================] - 2s 20ms/step - loss: 0.2506 - accuracy: 0.8962 - auc: 0.9476 - val_loss: 0.2450 - val_accuracy: 0.8970 - val_a

In [142]:
# reduce dims of test_pred_class
# stft_eval_dict['test_pred_class'] = stft_eval_dict['test_pred_class'].reshape(-1)
np.asarray(stft_eval_dict["true_labels"]).shape


(38, 1)

In [140]:
print(metrics.classification_report(np.asarray(stft_eval_dict["true_labels"]), stft_eval_dict["test_pred_class"]))

ValueError: Found input variables with inconsistent numbers of samples: [38, 1185]

In [122]:
stft_eval_dict["true_labels"].shape

(32, 1)

In [110]:
mel_model, mel_history, mel_eval_dict = train_model_pipeline('data/organized_spectrograms/Mel', lr=0.005)

Found 3951 files belonging to 2 classes.
Using 2766 files for training.
Found 3951 files belonging to 2 classes.
Using 1185 files for validation.
Epoch 1/100
87/87 [==============================] - 4s 41ms/step - loss: 0.4799 - accuracy: 0.7487 - auc: 0.7992 - val_loss: 0.4276 - val_accuracy: 0.7671 - val_auc: 0.8029
Epoch 2/100
23/87 [======>.......................] - ETA: 1s - loss: 0.4319 - accuracy: 0.7649 - auc: 0.7814

KeyboardInterrupt: 

In [95]:
logmel_model, logmel_history, logmel_eval_dict = train_model_pipeline('data/organized_spectrograms/LogMel', lr=0.005)

Found 3951 files belonging to 2 classes.
Epoch 1/100
86/86 [==============================] - 4s 30ms/step - loss: 0.2482 - accuracy: 0.7162 - auc: 0.8051 - precision_22: 0.7162 - recall_22: 0.7162 - val_loss: 0.5514 - val_accuracy: 0.7005 - val_auc: 0.7749 - val_precision_22: 0.7005 - val_recall_22: 0.7005
Epoch 2/100
86/86 [==============================] - 3s 24ms/step - loss: 0.2087 - accuracy: 0.7664 - auc: 0.8454 - precision_22: 0.7664 - recall_22: 0.7664 - val_loss: 0.4127 - val_accuracy: 0.7734 - val_auc: 0.8624 - val_precision_22: 0.7734 - val_recall_22: 0.7734
Epoch 3/100
86/86 [==============================] - 3s 24ms/step - loss: 0.1988 - accuracy: 0.7737 - auc: 0.8574 - precision_22: 0.7737 - recall_22: 0.7737 - val_loss: 0.4420 - val_accuracy: 0.7734 - val_auc: 0.8524 - val_precision_22: 0.7734 - val_recall_22: 0.7734
Epoch 4/100
86/86 [==============================] - 3s 27ms/step - loss: 0.2017 - accuracy: 0.7711 - auc: 0.8494 - precision_22: 0.7711 - recall_22: 0.771

In [103]:
stft_eval_dict.keys()
# print len of each key
for key in stft_eval_dict.keys():
    print(key, len(stft_eval_dict[key]))


test_proba 448
test_preds 431
true_labels 448


In [102]:
assert len(stft_eval_dict["test_proba"]) == len(stft_eval_dict["test_preds"]) == len(stft_eval_dict["true_labels"])
metrics.classification_report(stft_eval_dict["true_labels"], stft_eval_dict["test_preds"], output_dict=True)


AssertionError: 